In [1]:
import os
import json, sys
from pprint import pprint
from botocore.exceptions import NoCredentialsError, ClientError
import logging
from tqdm import tqdm
import pandas as pd
import numpy as np


In [2]:
#  just change these values
sub_root_dir = r"C:\Users\user\Downloads\textract\subjects"
scans_root_dir = r'C:\Users\user\Downloads\textract\scans'
sub_codes = ['KCE051', 'KCE055', 'KCE502', 'KCE503', 'KNC501']
scan_img_file_type = '.png'

bucket_name = 'process-question-answer-scans'
region_name='ap-south-1'
aws_access_key_id = "hakdugbua"  # access to "diabetic-retinopathy-data-from-radiology" with read, write and list access
aws_secret_access_key = "hiaduhgiuadh"

In [3]:
%%time
import boto3
import botocore
from botocore.exceptions import ClientError
from collections import defaultdict

def get_all_s3_keys(bucket, subdirectory, file_type=''):
    keys = []
    kwargs = {'Bucket': bucket, 'Prefix': subdirectory}
    s3_client = boto3.client('s3',
                         region_name=region_name,
                         aws_access_key_id=aws_access_key_id,
                         aws_secret_access_key=aws_secret_access_key)
    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.endswith(file_type):
                keys.append(key)
        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break
    return keys

def upload_to_s3(channel, file): 
    s3 = boto3.resource('s3',
                         region_name=region_name,
                         aws_access_key_id=aws_access_key_id,
                         aws_secret_access_key=aws_secret_access_key) 
    data = open(file, "rb")
    key = channel + '/' + file
    print("Uploading file {} to s3://{}/{}".format(file, bucket, channel))
    s3.Bucket(bucket).put_object(Key=key, Body=data)
    
def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

def upload_dir_to_s3(bucket, s3_folder, dir_to_upload):
    s3_client = boto3.client('s3',
                             region_name=region_name,
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key)
#     print("Uploading {} to s3://{}/{}".format(dir_to_upload, bucket, s3_folder))
    # enumerate local files recursively
    for root, dirs, files in os.walk(dir_to_upload):
        for filename in tqdm(files):
            # construct the full local path
            local_path = os.path.join(root, filename)
            # construct the full Dropbox path
            relative_path = os.path.relpath(local_path, dir_to_upload)
            s3_path = os.path.join(s3_folder, relative_path).replace("\\", "/")
            try:
                s3_client.head_object(Bucket=bucket, Key=s3_path)
                print("Path found on S3! Deleting %s..." % s3_path)
                try:
                    s3_client.delete_object(Bucket=bucket, Key=s3_path)
                    try:
#                         print("Uploading {} to s3://{}/{}".format(dir_to_upload, bucket, s3_path)
                        s3_client.upload_file(local_path, Bucket=bucket, Key=s3_path)
                    except ClientError as e:
                        logging.error(e)
                except:
                    print("Unable to delete from s3 %s..." % s3_path)
            except:
                try:
                    s3_client.upload_file(local_path, Bucket=bucket, Key=s3_path)
                except ClientError as e:
                    logging.error(e)
    print(f"Upload completed successfully to s3://{bucket}/{s3_folder}")
    
def download_dir(s3_folder, local_path, bucket=""):
    client = boto3.client('s3', region_name=region_name)
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket': bucket,
        'Prefix': s3_folder,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local_path, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    print("{} files found in {} directories. Downloading now...".format(len(keys), len(dirs)))
    for k in tqdm(keys):
        dest_pathname = os.path.join(local_path, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        try:
#             print("Downloading {}".format(dest_pathname))
            client.download_file(bucket, k, dest_pathname)
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print("The object does not exist.")
            else:
                raise
    print("{} files downloaded successfully.".format(len(keys)))
def download_from_s3(s3_filename, local_path="0.fbx"):
    s3_client = boto3.client('s3',
                             region_name=region_name,
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key)
    print("Downloading file {} to {}".format(s3_filename, local_path))
    try:
        s3_client.download_file(bucket, Key=s3_filename, Filename=local_path)
        # it waits till the download completes then moves the execution to forward
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise
# upload_to_s3("pretrained", 'se_resnext50_32x4d-a260b3a4.pth')
# upload_dir_to_s3(bucket, s3_folder, dir_to_upload)

# download('http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth')
# download_dir(s3_folder='aptos-2015/test_images_768/', local_path='/home/ec2-user/SageMaker/data/aptos-2015/test_images_768/', bucket=bucket)
# download_from_s3(s3_filename='pytorch-training-2020-12-29-09-38-13-247/source/sourcedir.tar.gz', local_path="/home/ec2-user/SageMaker/checkpoint/sourcedir.tar.gz")


Wall time: 5.08 s


### Uploading scanned photos files to s3 for textraction

### Remove initial pages that contains introdcution, unless it creates the noise

In [4]:
for sub_code in sub_codes:
    dir_to_upload = os.path.join(scans_root_dir, sub_code)
    if(os.path.isdir(dir_to_upload)):
        upload_dir_to_s3(bucket_name, s3_folder=sub_code, dir_to_upload=dir_to_upload)
    else:
        print(f"{dir_to_upload} doesn't exists")

  0%|                                                                                                                        | 0/198 [00:00<?, ?it/s]

Upload completed successfully to s3://process-question-answer-scans/KCE051


  0%|                                                                                                                        | 0/195 [00:00<?, ?it/s]

Upload completed successfully to s3://process-question-answer-scans/KCE055


  0%|                                                                                                                        | 0/170 [00:00<?, ?it/s]

Upload completed successfully to s3://process-question-answer-scans/KCE502


  0%|                                                                                                                         | 0/71 [00:00<?, ?it/s]

Upload completed successfully to s3://process-question-answer-scans/KCE503


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71/71 [22:06<00:00, 18.69s/it]

Upload completed successfully to s3://process-question-answer-scans/KNC501


In [5]:
def textraction(sub_code):
    lines = []
    files = []   
    textract = boto3.client(service_name='textract', 
                            region_name=region_name,
                            aws_access_key_id=aws_access_key_id,
                            aws_secret_access_key=aws_secret_access_key)
    try:
        files = get_all_s3_keys(bucket_name, subdirectory=sub_code, file_type=scan_img_file_type)
        if files == []:
            print(f'No file found in s3://{bucket_name}/{sub_code}, Upload scans first.')
            return
    except Exception as e:
        print("Caught Exception", e)
        return
    
    txt_file_path = os.path.join(sub_root_dir, sub_code)+'.txt'
    txt_file_opened = open(txt_file_path, 'w')

    # detecting text in the documents and writing to txt
    for s3_file_path in tqdm(files):
        try:
            response = textract.detect_document_text(
                Document={
                    "S3Object": {
                        "Bucket": bucket_name,
                        "Name": s3_file_path
                    }
                }
            )
            for item in response["Blocks"]:
                if item["BlockType"] == "LINE":
                    txt_file_opened.write(item["Text"]+'\n')                
        except Exception as e:
            print(e)
    txt_file_opened.close()

In [6]:
for sub_code in sub_codes:
    textraction(sub_code=sub_code)

 51%|████████████████████████████████████████████████████████▏                                                      | 86/170 [05:46<06:57,  4.97s/it]

'charmap' codec can't encode character '\u20b9' in position 40: character maps to <undefined>



 51%|████████████████████████████████████████████████████████▊                                                      | 87/170 [05:51<06:44,  4.88s/it]

'charmap' codec can't encode character '\u20b9' in position 19: character maps to <undefined>


 86%|██████████████████████████████████████████████████████████████████████████████████████████████▍               | 146/170 [14:08<01:47,  4.48s/it]

...

 88%|████████████████████████████████████████████████████████████████████████████████████████████████▍             | 149/170 [14:21<01:28,  4.19s/it]

'charmap' codec can't encode character '\u20b9' in position 21: character maps to <undefined>


 89%|██████████████████████████████████████████████████████████████████████████████████████████████████▎           | 152/170 [14:31<01:05,  3.64s/it]

'charmap' codec can't encode character '\u20b9' in position 10: character maps to <undefined>


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71/71 [14:42<00:00, 12.43s/it]


In [94]:
# def quanify(sub_code, sub_name, drop_sem_str, isCamScanner):
#     df = pd.DataFrame({"question1": [''],
#                        'question2': [''],
#                        'answer': ['']
#                        })  
#     txt_file_path = os.path.join(sub_root_dir, sub_code)+'.txt'

#     with open(txt_file_path, 'r') as file:
#         lines = file.readlines()

#         contents = []
#         # if a line conatains any of these, remove that
#         drop_strs = ['QUESTIONS-ANSWERS', 'LONG ANSWER TYPE AND MEDIUM ANSWER TYPE QUESTIONS',
#                     'VERY IMPORTANT QUESTIONS', 'FOLLOWING QUESTIONS ARE VERY IMPORTANT. THESE QUESTIONS',
#                     'MAY BE ASKED IN YOUR SESSIONALS AS WELL AS', 'UNIVERSITY EXAMINATION',
#                     'MARKS', '2 MARKS QUESTIONS', 'QUESTION', 
#                     'PART-1', 'PART-2', 'PART-3', 'PART-4', 'PART-5', 'PART-6', 'PART-7', 'PART-8', 'PART-9', 'PART-10', 
#                     'PART-11', 'PART-12', 'PART-13', 'PART-14', 'PART-15', 'PART-16', 'PART-17', 'PART-18', 'PART-19', 'PART-20',
#                     'PART-21', 'PART-22', 'PART-23', 'PART-24', 'PART-25', 'PART-26', 'PART-27', 'PART-28', 'PART-29', 'PART-30',
#                     'AKTU', 'MARKS 01', 'MARKS 02', 'MARKS 05', 'MARKS 10', 'MARKS 15', 
#                     '2010-11,', '2011-12,', '2012-13,', '2013-14,', '2014-15,', '2015-16,', '2016-17,', '2017-18,', '2018-19,', 
#                     '2019-20,', '2020-21,', '2021-22,', '2022-23,']

#         question_end = ['.', '?', ',', '!']
#         drop_after_qa_detn = ['QUE ', 'OR', 'ANSWER']  # Leading part of the Que line will also be removed.
#         lines1 = []
#         que1_line_start, que1_line_end, que2_line_start, que2_line_end, ans_line_start, ans_line_end = 0, 0, 0, 0, 0, 0
#         que1_found, que2_found, ans_found = 0, 0, 0
#         que_idx, ans_idx = 0, 0

#         for line in lines:
#             line = line.split('\n')[0].upper()
#             lines1.append(line)

#         # appending all the dropping strings to one list
#         drop_strs.append(drop_sem_str)
#         if isCamScanner:
#             drop_strs.append('SCANNED WITH CAMSCANNER')
#         drop_strs.append(sub_name)
#         for w in contents: drop_strs.append(w)

#         lines = []
        
#         # removing all the sub-strings
#         for line in lines1:
#             found = 0
#             for drop_str in drop_strs:
#                 if line.find(drop_str) != -1: found = 1
#             if found != 1: lines.append(line)
#         del lines1
        
#         with open(os.path.join(sub_root_dir, "txt_file_path.txt"), 'w') as file:
#             for line in lines:
#                 file.writelines(line+"\n")
        
#         start = True
#         for i, line in enumerate(lines):
#             if line.find('QUE') != -1 or line.find('QVE') != -1:
#                 if start:
#                     que1_found = 1
#                     que1_line_start = i
#                 else:
#                     ans_line_end = i - 1
#                     ###################### put the content in the dataframe
#                     if (que1_found ==1) and (ans_found==1):
#                         ###########  question1
#                         question = lines[que1_line_start + 1]
#                         if que1_line_start < que1_line_end:
#                             for q in range(que1_line_end - que1_line_start - 1):
#                                 que1_line_start += 1
#                                 question += ' ' + lines[que1_line_start + 1]
#                         que1_found = 0 # not set it here
#                         df.at[que_idx, 'question1'] = question
                        
#                         if que2_found==1:
#                             ###########  question2
#                             question = lines[que2_line_start + 1]
#                             if que2_line_start < que2_line_end:
#                                 for q in range(que2_line_end - que2_line_start - 1):
#                                     que2_line_start += 1
#                                     question += ' ' + lines[que2_line_start + 1]
#                             que2_found = 0
#                             df.at[que_idx, 'question2'] = question
                        
#                         ########### answer
#                         answer = lines[ans_line_start + 1]
#                         if ans_line_start < ans_line_end:
#                             for a in range(ans_line_end - ans_line_start - 1):
#                                 ans_line_start += 1
#                                 answer += ' ' + lines[ans_line_start + 1]
#                         ans_found = 0
#                         df.at[ans_idx, 'answer'] = answer
#                         que_idx += 1
#                         ans_idx += 1

#                     que1_found = 1
#                     que1_line_start = i
#                 continue

#             elif line == 'OR':
#                 que1_line_end = i-1
#                 que2_line_start = i
#                 que2_found = 1
#                 continue

#             elif line.find('ANSWER') != -1 or line == 'ANSWER':
#                 if que2_found ==1:
#                     que2_line_end = i - 1
#                 elif que1_found==1:
#                     que1_line_end=i-1
#                 ans_line_start = i
#                 ans_found = 1
#                 start=False
#                 continue  
#     return df

## QUANIFY

In [95]:
# sub_codes = ['KCE501', 'KCE051', 'KCE055', 'KCE502', 'KCE503', 'KNC501']
# sub_names = ['GEOTECHNICAL ENGINEERING', 'CONCRETE TECHNOLOGY', 'INTRODUCTION TO HYDROLOGY', 
#             'STRUCTURAL ANALYSIS', 'QUANTITY ESTIMATION & MANAGEMENT', 'CONSTITUTION OF INDIA, LAW & ENGINEERING']
# drop_sem_strs = ['CE-SEM-5', 'CE-SEM-5', 'CE-SEM-5', 'CE-SEM-4', 'CE-SEM-5', 'NCC-SEM-5 & 6']

# for sub_code, sub_name, drop_sem_str in zip(sub_codes, sub_names, drop_sem_strs):
# #     if sub_code=='KCE501':
# #         continue
#     df = quanify(sub_code, sub_name, drop_sem_str, isCamScanner=True)
    
#     csv_file_path = os.path.join(sub_root_dir, sub_code)+'.csv'
#     df.to_csv(csv_file_path, index_label='id', sep='|')
    
#     print('QA csv file created: '+ csv_file_path)

QA csv file created: C:\Users\user\Downloads\textract\subjects\KCE501.csv
QA csv file created: C:\Users\user\Downloads\textract\subjects\KCE051.csv
QA csv file created: C:\Users\user\Downloads\textract\subjects\KCE055.csv
QA csv file created: C:\Users\user\Downloads\textract\subjects\KCE502.csv
QA csv file created: C:\Users\user\Downloads\textract\subjects\KCE503.csv
QA csv file created: C:\Users\user\Downloads\textract\subjects\KNC501.csv


In [95]:
# import difflib
# a = 'WHAT ARE THE ADVANTAGES AND DISADVANTAGES OF STEEL USED AS A STRUCTURAL MATERIAL ?'
# b = 'WRITE ADVANTAGES OF STEEL USED AS A STRUCTURAL MATERIAL.'
# seq = difflib.SequenceMatcher()
# seq.set_seqs(a.lower(), b.lower()) # string b is x times similar to string a 
# d = seq.ratio()*100
# print(d)


# from difflib import get_close_matches

# word_list =['fjadbbiu', 'jdbfag', 'nsfkubusi', 'bdjfh','gvhvh', 'nfd', 'njfd', a]
# matches = get_close_matches('abcd', word_list, n=2, cutoff=0.1)
# print(matches)


# seq = difflib.SequenceMatcher()
# seq.set_seqs('nsfkubusi', 'njfdu') # string b is x times similar to string a 
# d = seq.ratio()*100
# print(d)


# from fuzzywuzzy import process

# q1_from_s3, similarity = process.extractOne(b, word_list)


76.81159420289855


In [96]:
def quanify(sub_code, sub_name, drop_sem_str, isCamScanner):
    df = pd.DataFrame({"question1": [''],
                       'question2': [''],
                       'answer': ['']
                       })  
    txt_file_path = os.path.join(sub_root_dir, sub_code)+'.txt'

    with open(txt_file_path, 'r') as file:
        lines = file.readlines()

        contents = []
        # if a line conatains any of these, remove that
        drop_strs = ['QUESTIONS-ANSWERS', 'LONG ANSWER TYPE AND MEDIUM ANSWER TYPE QUESTIONS',
                    'VERY IMPORTANT QUESTIONS', 'FOLLOWING QUESTIONS ARE VERY IMPORTANT. THESE QUESTIONS',
                    'MAY BE ASKED IN YOUR SESSIONALS AS WELL AS', 'UNIVERSITY EXAMINATION',
                    'MARKS', '2 MARKS QUESTIONS', 'QUESTION', 
                    'PART-1', 'PART-2', 'PART-3', 'PART-4', 'PART-5', 'PART-6', 'PART-7', 'PART-8', 'PART-9', 'PART-10', 
                    'PART-11', 'PART-12', 'PART-13', 'PART-14', 'PART-15', 'PART-16', 'PART-17', 'PART-18', 'PART-19', 'PART-20',
                    'PART-21', 'PART-22', 'PART-23', 'PART-24', 'PART-25', 'PART-26', 'PART-27', 'PART-28', 'PART-29', 'PART-30',
                    'AKTU', 'MARKS 01', 'MARKS 02', 'MARKS 05', 'MARKS 10', 'MARKS 15', 
                    '2010-11,', '2011-12,', '2012-13,', '2013-14,', '2014-15,', '2015-16,', '2016-17,', '2017-18,', '2018-19,', 
                    '2019-20,', '2020-21,', '2021-22,', '2022-23,']

        question_end = ['.', '?', ',', '!']
        drop_after_qa_detn = ['QUE ', 'OR', 'ANSWER']  # Leading part of the Que line will also be removed.
        que1_line_start, que1_line_end, que2_line_start, que2_line_end, ans_line_start, ans_line_end = 0, 0, 0, 0, 0, 0
        que1_found, que2_found, ans_found = 0, 0, 0
        que_idx, ans_idx = 0, 0

        lines1 = []
        for line in lines:
            line = line.split('\n')[0].upper()
            lines1.append(line)

        # appending all the dropping strings to one list
        drop_strs.append(drop_sem_str)
        drop_strs.append(sub_name)
        if isCamScanner:
            drop_strs.append('SCANNED WITH CAMSCANNER')
        for w in contents: drop_strs.append(w)
        
#         print(drop_strs)
#         import re
        lines = []
        for line in lines1:
            for drop_str in drop_strs:
                line = line.replace(drop_str, "")
#                 line = re.sub(' +', ' ', line)
                line = " ".join(line.split())  # removes more "  " from line string
            lines.append(line)
        del lines1
        
        with open(os.path.join(sub_root_dir, sub_code+"txt_file_path.txt"), 'w') as file:
            for line in lines:
                file.writelines(line+"\n")
        
        start = True
        for i, line in enumerate(lines):
            if line.find('QUE') != -1 or line.find('QVE') != -1:
                if start:
                    que1_found = 1
                    que1_line_start = i
                else:
                    ans_line_end = i - 1
                    ###################### put the content in the dataframe
                    if (que1_found ==1) and (ans_found==1):
                        ###########  question1
                        question = lines[que1_line_start + 1]
                        if que1_line_start < que1_line_end:
                            for q in range(que1_line_end - que1_line_start - 1):
                                que1_line_start += 1
                                question += ' ' + lines[que1_line_start + 1]
                        que1_found = 0 # not set it here
                        df.at[que_idx, 'question1'] = question
                        
                        if que2_found==1:
                            ###########  question2
                            question = lines[que2_line_start + 1]
                            if que2_line_start < que2_line_end:
                                for q in range(que2_line_end - que2_line_start - 1):
                                    que2_line_start += 1
                                    question += ' ' + lines[que2_line_start + 1]
                            que2_found = 0
                            df.at[que_idx, 'question2'] = question
                        
                        ########### answer
                        answer = lines[ans_line_start + 1]
                        if ans_line_start < ans_line_end:
                            for a in range(ans_line_end - ans_line_start - 1):
                                ans_line_start += 1
                                answer += ' ' + lines[ans_line_start + 1]
                        ans_found = 0
                        df.at[ans_idx, 'answer'] = answer
                        que_idx += 1
                        ans_idx += 1

                    que1_found = 1
                    que1_line_start = i
                continue

            elif line == 'OR':
                que1_line_end = i-1
                que2_line_start = i
                que2_found = 1
                continue

            elif line.find('ANSWER') != -1 or line == 'ANSWER':
                if que2_found ==1:
                    que2_line_end = i - 1
                elif que1_found==1:
                    que1_line_end=i-1
                ans_line_start = i
                ans_found = 1
                start=False
                continue  
                
        ###########  question1
        question = lines[que1_line_start + 1]
        if que1_line_start < que1_line_end:
            for q in range(que1_line_end - que1_line_start - 1):
                que1_line_start += 1
                question += ' ' + lines[que1_line_start + 1]
        que1_found = 0 # not set it here
        df.at[que_idx, 'question1'] = question

        if que2_found==1:
            ###########  question2
            question = lines[que2_line_start + 1]
            if que2_line_start < que2_line_end:
                for q in range(que2_line_end - que2_line_start - 1):
                    que2_line_start += 1
                    question += ' ' + lines[que2_line_start + 1]
            que2_found = 0
            df.at[que_idx, 'question2'] = question

        ########### answer
        answer = lines[ans_line_start + 1]
        if ans_line_start < ans_line_end:
            for a in range(ans_line_end - ans_line_start - 1):
                ans_line_start += 1
                answer += ' ' + lines[ans_line_start + 1]
        ans_found = 0
        df.at[ans_idx, 'answer'] = answer
            
    return df

In [97]:
sub_codes = ['KCE501', 'KCE051', 'KCE055', 'KCE502', 'KCE503', 'KNC501']
sub_names = ['GEOTECHNICAL ENGINEERING', 'CONCRETE TECHNOLOGY', 'INTRODUCTION TO HYDROLOGY', 
            'STRUCTURAL ANALYSIS', 'QUANTITY ESTIMATION & MANAGEMENT', 'CONSTITUTION OF INDIA, LAW & ENGINEERING']
drop_sem_strs = ['CE-SEM-5', 'CE-SEM-5', 'CE-SEM-5', 'CE-SEM-4', 'CE-SEM-5', 'NCC-SEM-5 & 6']

for sub_code, sub_name, drop_sem_str in zip(sub_codes, sub_names, drop_sem_strs):
#     if sub_code=='KCE501':
#         continue
    df = quanify(sub_code, sub_name, drop_sem_str, isCamScanner=True)
    
    csv_file_path = os.path.join(sub_root_dir, sub_code)+'.csv'
    df.to_csv(csv_file_path, index_label='id', sep='|')
    
    print('QA csv file created: '+ csv_file_path)

QA csv file created: C:\Users\user\Downloads\textract\subjects\KCE501.csv
QA csv file created: C:\Users\user\Downloads\textract\subjects\KCE051.csv
QA csv file created: C:\Users\user\Downloads\textract\subjects\KCE055.csv
QA csv file created: C:\Users\user\Downloads\textract\subjects\KCE502.csv
QA csv file created: C:\Users\user\Downloads\textract\subjects\KCE503.csv
QA csv file created: C:\Users\user\Downloads\textract\subjects\KNC501.csv
